# Machine Learning

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.decomposition import PCA
import random
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Load saved data pickle: 
with open('filtered9b.pickle', 'rb') as handle:
    data_dict = pickle.load(handle)
    
print(len(data_dict))

with open('download_meta9b.pickle', 'rb') as handle:
    df_download = pickle.load(handle)
    
print(len(data_dict))
#print(data_dict)
df_download.head()

In [ ]:
# DONE in downloader
column_values = df_download[["Make_normalized", "Model_normalized"]].values.ravel()
unique_values =  pd.unique(column_values)

In [ ]:
# done in downloader
makes = pd.unique(df_download[["Make_normalized"]].values.ravel())
print(makes)
df_download["unique_id"] = -1
c = 0
for make in makes: 
    sub_df = df_download[df_download["Make_normalized"] == make]
    print(make)
    models = pd.unique(sub_df[["Model_normalized"]].values.ravel())
    for model in models: 
        print("     -", model)
        #test_df = df_download[(df_download["Make_normalized"] == make and df_download["Model_normalized"] == model)]
        #df_download["unique_id"] = np.where(((df_download["Make_normalized"] == make) & (df_download["Model_normalized"] == model)),c)
        #, 'unique_id'] = c
        df_download.loc[(df_download['Make_normalized'] == make) & (df_download['Model_normalized'] == model), 'unique_id'] = c
        c+=1        
    

In [ ]:
# I think this is just some test i forgot to delete 
print(df_download["unique_id"].value_counts())
random_tag = random.choice(list(df_download['unique_id'].unique()))

train = df_download[df_download['unique_id']!=random_tag]
test = df_download[df_download['unique_id'] == random_tag]
train.info()

In [ ]:
# Now this is an important part 
# Basicly we iterate over the dictionary and construct a pandas dataframe 

all_tags = dict()
mean_tags = list()
models = dict()
c = 0

df = pd.DataFrame([], dtype=int)

st = time.time()
for key in data_dict: 
    # Key is the url link of the image that we use for indexing
    
    # Create index
    df = df.append(pd.Series(name=key))
    c_tags = 0
    
    # Lets iterate over tags that are giver for the image 
    for tag_id in data_dict[key]:
        c_tags += 1
        
        if tag_id not in all_tags: 
            all_tags[tag_id] = 1
        all_tags[tag_id] += 1
        
        # Here we get the tag name (which we dont need/use)
        tag = TAGS.get(tag_id, tag_id)
        
        data = data_dict[key].get(tag_id)
        if isinstance(data, bytes):
            try: 
                data = data.decode('utf-8', 'ignore').encode("utf-8")
            except: 
                pass
        try: 
            
            if str(tag_id) in df.columns: 
                # if tag_ id is already in columns - we just insert the values 
                df.loc[key, str(tag_id)] = data
            else: 
                # Else we have to add another column with tag_id 
                df[str(tag_id)] = np.nan   
                df.loc[key, str(tag_id)] = str(data)

            
        except Exception as e: 
            pass
            #print(data)
            #print(e)
            #print(f"{tag:25}: {data}")
     
    mean_tags.append(c_tags)
    
et = time.time()

print("Median number of tags per imamge:", np.mean(np.array(mean_tags)))
print("Consumed time:", et-st)

In [ ]:
# Statistics for nerds
df.info()
# Also get percentage of missing values. 
per_missing = (df.isnull().sum()/len(df)).mean()
print("Percentage of missing values is:", per_missing)

In [ ]:
# With this we create similar dataset that has same shape, but has 1 instead of value and 0 instead of None
df2 = df.notnull().astype('int')
df2.info()
df2.head()

In [ ]:
try: 
    df2 = df2.drop(list(one_hot), axis=1)
    
except:
    pass
df2.index.names = ['url_o']
df2 = df2.join(dftest)
#df2.merge(dftest, how='left')
#df2.index.names = ['url_o']
df2.head()



In [ ]:
import random
print(df_download["type"].value_counts())

def generate_random():
    
    random_tag = random.choice(list(df_download['unique_id'].unique()))
    print("generating random:", random_tag)
    return random_tag
    

In [ ]:
def learn(traindata_X, traindata_Y):
    # Import the model we are using
    from sklearn.ensemble import RandomForestRegressor
    # Instantiate model with decision trees
    rf = RandomForestRegressor(random_state = 42)
    # Train the model on training data
    rf.fit(traindata_X, traindata_Y);
    return rf

In [ ]:
def get_solution(testdata_Y):
    
    testdata_Y.head()
    solution = -1
    row = testdata_Y.iloc[0] # first row of data frame (Aleshia Tomkiewicz)
    for i in range(len(targets)): 
        target = targets[i]
        if testdata_Y[target].any() == 1:
            solution = i
    #print("Getting the solution:", solution)
    return solution


In [ ]:
# Use the forest's predict method on the test data
def predict(rf, testdata_X):
    predictions = np.rint(rf.predict(testdata_X))
    res = []
    for row in predictions: 
        res.append(np.where(row == np.amax(row))[0][0])
    #print("Predicting:", res)
    return res

In [ ]:
list(traindata_X.columns)
test.head()

In [ ]:
# In this part the main machine leadning happens 

# Set number of iterations: 
num_iter = 50
tr = [0,0]
all_sol = []
targets = ['Apple','Canon','Fujifilm','Kodak','Nikon','Olympus','Panasonic','Samsung','Sony']
features = df2.columns.difference(targets)
all_res = []

# We repeat learning for given number of iterations 
for i in range(num_iter):
    tag = generate_random()
    # Split training and test data based on random tag
    train = df2[df2['unique_id']!=tag]
    test = df2[df2['unique_id'] == tag]
    
    
    traindata = train
    testdata = test
    
    #Drop rows that have information about models
    try : 
        traindata = traindata.drop(['unique_id'], axis=1)
    except: 
        pass
    try : 
        testdata = testdata.drop(['unique_id'], axis=1)
    except: 
        pass
    
    # Split features and targets
    traindata_X = traindata[traindata.columns.difference(targets)]
    traindata_Y = traindata[targets]
    
    testdata_X = testdata[testdata.columns.difference(targets)]
    testdata_Y = testdata[targets]
    
    solution = get_solution(testdata_Y)
    
    all_sol.append(solution)
    # Here the Random forrest classifier will learn 
    rf = learn(traindata_X, traindata_Y)
    # And predict
    res = predict(rf, testdata_X)
    all_res.append(res)
    
    # Quick analysis
    for el in res: 
        if el == solution: 
            tr[0]+=1
        else: 
            tr[1]+=1
    print(tr)
print("FINISHED")
print("[CORRECT: FALIURES]")
print(tr)

In [ ]:
# Get feature importances
#1
import time
import numpy as np

start_time = time.time()
importances = rf.feature_importances_
std = np.std([
    tree.feature_importances_ for tree in rf.estimators_], axis=0)
elapsed_time = time.time() - start_time

import pandas as pd
forest_importances = pd.Series(importances, index=features)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
#Get feature importances
#2
best_attrs = []
from PIL.ExifTags import TAGS
#plt.barh(features, rf.feature_importances_)
best = rf.feature_importances_.argsort()[-10:][::-1]
for el in best: 
    tag = TAGS.get(int(features[el]))
    best_attrs.append(features[el])
    print(features[el], "(", tag, "):", rf.feature_importances_[el])
print("_____________")



In [ ]:
# PLOT and print missing values based on feature importances 

targets = ['Apple','Canon','Fujifilm','Kodak','Nikon','Olympus','Panasonic','Samsung','Sony']
import matplotlib.pyplot as plt

for col in best_attrs[:4]:
    fig = plt.figure(figsize=(8, 4))
    
    ax = fig.add_axes([0,0,1,1])
    ax.title.set_text("Tag_ID: "+ col +" - " + TAGS.get(int(col)))
    res = []
    for target in targets: 
        sub_df = df2.loc[df2[target] == 1]
        #print(sub_df.info())
        # Calculate percentage 
        sol = sub_df[col].astype(bool).sum(axis=0)*100/len(sub_df)
        print(target , ",", "target", sol)
        res.append(sol)
        
    
    ax.bar(targets,res)
    plt.show()
    

In [ ]:
# EDIT RESULTS? I dont remember. 
# it is inportant for the future
res_together = []
sol_together = []
for i in range(len(all_res)): 
    tmp_res = all_res[i]
    tmp_sol = all_sol[i]
    for j in range(len(tmp_res)):
        res_together.append(tmp_res[j])
        sol_together.append(tmp_sol)

#print(res_together)
#print()
#print(sol_together)

In [ ]:
# a basic confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
#plot_confusion_matrix(res_together, sol_together)  
cm = confusion_matrix(sol_together, res_together)
print(cm)

In [ ]:
# Confusion matriix with colors :) 
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay


ls = [0, 1] # your y labels
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ls)
plt.show()

In [ ]:
# We save the results
np.save("y_test.npy", np.array(sol_together))
np.save("pred.npy", np.array(res_together))


Another confusion matrix that was used in report has been done using: 
https://github.com/wcipriano/pretty-print-confusion-matrix